In [18]:
import warnings

warnings.filterwarnings("ignore")

In [19]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [20]:
from langchain_openai import ChatOpenAI
import os

os.environ['OPENAI_API_KEY'] = 'API_KEY'

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'stop', 'logprobs': None}, id='run-88366d68-399a-49a3-a824-34dae1ab62c5-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Agent = LLM + Tools

In [21]:
import subprocess
from langchain_core.tools import tool
import platform


@tool
def list_files():
    """List files in current directory"""
    os_name = platform.system()
    if os_name=='Linux':
        return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")
    if os_name=='Windows':
        return subprocess.run("dir/s".split(), stdout=subprocess.PIPE, shell=True).stdout.decode("cp850")

list_files.invoke({"input":""})

' El volumen de la unidad C es OS\r\n El número de serie del volumen es: CE4B-7E93\r\n\r\n Directorio de c:\\Users\\Denys\\Downloads\\vidhya-agents-course-main\\notebooks\r\n\r\n09/09/2024  08:05 a.\xa0m.    <DIR>          .\r\n09/09/2024  08:05 a.\xa0m.    <DIR>          ..\r\n10/30/2024  12:44 a.\xa0m.             9,120 1.0-langchain-basics.ipynb\r\n10/30/2024  12:43 a.\xa0m.            20,962 1.1-langchain-simple-chains.ipynb\r\n10/30/2024  12:50 a.\xa0m.            23,508 1.2-langchain-runnables-and-lcel.ipynb\r\n09/09/2024  08:05 a.\xa0m.            34,340 1.3-langchain-simple-agent.ipynb\r\n09/09/2024  08:05 a.\xa0m.           240,681 2.0-autogen-building-agents.ipynb\r\n09/09/2024  08:05 a.\xa0m.            56,378 2.1-autogen-building-research-assistant.ipynb\r\n09/09/2024  08:05 a.\xa0m.               882 3.0-langgraph-building-stateful-applications.ipynb\r\n09/09/2024  08:05 a.\xa0m.            24,171 3.1-langgraph-buid-research-assistant.ipynb\r\n09/09/2024  08:05 a.\xa0m.   

In [22]:
llm_with_tool = llm.bind_tools([list_files])

llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002117B9AD8B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002117B993940>, root_client=<openai.OpenAI object at 0x000002117B257CD0>, root_async_client=<openai.AsyncOpenAI object at 0x000002117B9AD910>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'list_files', 'description': 'List files in current directory', 'parameters': {'properties': {}, 'type': 'object'}}}]}, config={}, config_factories=[])

In [23]:
llm_with_tool.invoke("List files in current directory")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AQ6MnaJwlIYpxLKmeYY3L6Kx', 'function': {'arguments': '{}', 'name': 'list_files'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 42, 'total_tokens': 52, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bfc6a7dc2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-19c716fb-c7ee-4cb8-86e8-ba4049edbe2d-0', tool_calls=[{'name': 'list_files', 'args': {}, 'id': 'call_AQ6MnaJwlIYpxLKmeYY3L6Kx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 10, 'total_tokens': 52, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [24]:
dictionary_with_tool_names = {
    "list_files": list_files,
}

output = llm_with_tool.invoke("List files in current directory")

if output.tool_calls:
    for tool_call in output.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call['args']
        tool_func = dictionary_with_tool_names[tool_name]
        print(tool_func.invoke(tool_args))

 El volumen de la unidad C es OS
 El número de serie del volumen es: CE4B-7E93

 Directorio de c:\Users\Denys\Downloads\vidhya-agents-course-main\notebooks

09/09/2024  08:05 a. m.    <DIR>          .
09/09/2024  08:05 a. m.    <DIR>          ..
10/30/2024  12:44 a. m.             9,120 1.0-langchain-basics.ipynb
10/30/2024  12:43 a. m.            20,962 1.1-langchain-simple-chains.ipynb
10/30/2024  12:50 a. m.            23,508 1.2-langchain-runnables-and-lcel.ipynb
09/09/2024  08:05 a. m.            34,340 1.3-langchain-simple-agent.ipynb
09/09/2024  08:05 a. m.           240,681 2.0-autogen-building-agents.ipynb
09/09/2024  08:05 a. m.            56,378 2.1-autogen-building-research-assistant.ipynb
09/09/2024  08:05 a. m.               882 3.0-langgraph-building-stateful-applications.ipynb
09/09/2024  08:05 a. m.            24,171 3.1-langgraph-buid-research-assistant.ipynb
09/09/2024  08:05 a. m.            51,484 4.0-rag-agent-langgraph.ipynb
09/09/2024  08:05 a. m.            98,

Agent = LLM + tools

In [25]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [26]:
tools = [list_files]

agent = create_tool_calling_agent(llm, tools, prompt_agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
agent_executor.invoke({'input': "List files in current directory"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


 El volumen de la unidad C es OS
 El número de serie del volumen es: CE4B-7E93

 Directorio de c:\Users\Denys\Downloads\vidhya-agents-course-main\notebooks

09/09/2024  08:05 a. m.    <DIR>          .
09/09/2024  08:05 a. m.    <DIR>          ..
10/30/2024  12:44 a. m.             9,120 1.0-langchain-basics.ipynb
10/30/2024  12:43 a. m.            20,962 1.1-langchain-simple-chains.ipynb
10/30/2024  12:50 a. m.            23,508 1.2-langchain-runnables-and-lcel.ipynb
09/09/2024  08:05 a. m.            34,340 1.3-langchain-simple-agent.ipynb
09/09/2024  08:05 a. m.           240,681 2.0-autogen-building-agents.ipynb
09/09/2024  08:05 a. m.            56,378 2.1-autogen-building-research-assistant.ipynb
09/09/2024  08:05 a. m.               882 3.0-langgraph-building-stateful-applications.ipynb
09/09/2024  08:05 a. m.            24,171 3.1-langgraph-buid-research-assistant.ipynb
09/09/2024  08:05 a. m.           

{'input': 'List files in current directory',
 'output': 'Here are the files in the current directory (`c:\\Users\\Denys\\Downloads\\vidhya-agents-course-main\\notebooks`):\n\n1. **1.0-langchain-basics.ipynb** - 9,120 bytes\n2. **1.1-langchain-simple-chains.ipynb** - 20,962 bytes\n3. **1.2-langchain-runnables-and-lcel.ipynb** - 23,508 bytes\n4. **1.3-langchain-simple-agent.ipynb** - 34,340 bytes\n5. **2.0-autogen-building-agents.ipynb** - 240,681 bytes\n6. **2.1-autogen-building-research-assistant.ipynb** - 56,378 bytes\n7. **3.0-langgraph-building-stateful-applications.ipynb** - 882 bytes\n8. **3.1-langgraph-buid-research-assistant.ipynb** - 24,171 bytes\n9. **4.0-rag-agent-langgraph.ipynb** - 51,484 bytes\n10. **4.1-autogen-multi-agent-workflows.ipynb** - 98,267 bytes\n11. **agentic-rag-langgraph.ipynb** - 51,281 bytes\n12. **Autogen Demo Notebook.ipynb** - 18,744 bytes\n13. **Tool Use.ipynb** - 10,394 bytes\n\nAdditionally, there is a directory named **assets-resources** which contai

In [28]:
agent_executor.invoke({'input': "What kind of files are in the current directory?"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


 El volumen de la unidad C es OS
 El número de serie del volumen es: CE4B-7E93

 Directorio de c:\Users\Denys\Downloads\vidhya-agents-course-main\notebooks

09/09/2024  08:05 a. m.    <DIR>          .
09/09/2024  08:05 a. m.    <DIR>          ..
10/30/2024  12:44 a. m.             9,120 1.0-langchain-basics.ipynb
10/30/2024  12:43 a. m.            20,962 1.1-langchain-simple-chains.ipynb
10/30/2024  12:50 a. m.            23,508 1.2-langchain-runnables-and-lcel.ipynb
09/09/2024  08:05 a. m.            34,340 1.3-langchain-simple-agent.ipynb
09/09/2024  08:05 a. m.           240,681 2.0-autogen-building-agents.ipynb
09/09/2024  08:05 a. m.            56,378 2.1-autogen-building-research-assistant.ipynb
09/09/2024  08:05 a. m.               882 3.0-langgraph-building-stateful-applications.ipynb
09/09/2024  08:05 a. m.            24,171 3.1-langgraph-buid-research-assistant.ipynb
09/09/2024  08:05 a. m.           

{'input': 'What kind of files are in the current directory?',
 'output': 'In the current directory, there are several Jupyter Notebook files and a couple of directories. Here’s a summary of the files:\n\n### Jupyter Notebook Files:\n1. **1.0-langchain-basics.ipynb** - 9,120 bytes\n2. **1.1-langchain-simple-chains.ipynb** - 20,962 bytes\n3. **1.2-langchain-runnables-and-lcel.ipynb** - 23,508 bytes\n4. **1.3-langchain-simple-agent.ipynb** - 34,340 bytes\n5. **2.0-autogen-building-agents.ipynb** - 240,681 bytes\n6. **2.1-autogen-building-research-assistant.ipynb** - 56,378 bytes\n7. **3.0-langgraph-building-stateful-applications.ipynb** - 882 bytes\n8. **3.1-langgraph-buid-research-assistant.ipynb** - 24,171 bytes\n9. **4.0-rag-agent-langgraph.ipynb** - 51,484 bytes\n10. **4.1-autogen-multi-agent-workflows.ipynb** - 98,267 bytes\n11. **agentic-rag-langgraph.ipynb** - 51,281 bytes\n12. **Autogen Demo Notebook.ipynb** - 18,744 bytes\n13. **Tool Use.ipynb** - 10,394 bytes\n\n### Directories:

In [29]:
import os

os.environ["TAVILY_API_KEY"] = 'API_KEY'

In [37]:
!pip install -qU langchain_community tavily-python

In [43]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults

In [44]:
search_tool = TavilySearchResults(
    max_results = 5,
    search_depth="advanced",
    include_answer=True,
)

search_tool.invoke('What is the capital of France?')

[{'url': 'https://www.britannica.com/facts/Paris',
  'content': 'Paris is the capital of France, located in the north-central part of the country. It is a major centre of commerce, culture, and tourism, with famous landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame de Paris.'},
 {'url': 'https://simple.wikipedia.org/wiki/Paris',
  'content': 'Events[change | change source]\nRelated pages[change | change source]\nReferences[change | change source]\nOther websites[change | change source]\nAmsterdam, Netherlands\xa0·\nAthens, Greece\xa0·\nBerlin, Germany\xa0·\nBratislava, Slovakia\xa0·\nBrussels, Belgium\xa0·\nBucharest, Romania\xa0·\nBudapest, Hungary\xa0·\nCopenhagen, Denmark\xa0·\nDublin, Republic of Ireland\xa0·\nHelsinki, Finland\xa0·\nLisbon, Portugal\xa0·\nLjubljana, Slovenia\xa0·\nLuxembourg City, Luxembourg\xa0·\nMadrid, Spain\xa0·\nNicosia, Cyprus1\xa0·\nParis, France\xa0·\nPrague, Czech Republic\xa0·\nRiga, Latvia\xa0·\nRome, Italy\xa0·\nSofia, Bulgaria\xa0·\

In [45]:
@tool
def create_file(file_path: str, content: str):
    """Create a file with the given content"""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"File created at {file_path} with content: {content}"

tools = [create_file, search_tool]

In [46]:
simple_research_agent = create_tool_calling_agent(llm, tools, prompt_agent)

simple_research_agent_executor = AgentExecutor(agent=simple_research_agent, tools=tools, verbose=True)

simple_research_agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [47]:
from IPython.display import Markdown
output_report = simple_research_agent_executor.invoke({'input': 'Write a summarized report about the main highlights of the Paris Olympics 2024 and save it locally as report-olympics-2024.md'})

Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 highlights'}`


[{'url': 'https://olympics.com/en/paris-2024/videos/list', 'content': 'From basketball to tennis, discover the full list of videos of the Paris 2024 Olympic Games (Jul 26-Aug 11, 2024). ... Experience the excitement of the Paris 2024 Games with daily sports highlights that capture the most thrilling moments from each day of competition. Relive iconic victories, unforgettable performances, and the intensity of the'}, {'url': 'https://www.forbes.com/sites/jackmagruder/2024/08/14/the-10-most-incredible-highlights-from-the-2024-paris-olympics/', 'content': 'Explore the 10 most incredible highlights from the 2024 Paris Olympics, including record-breaking moments and inspiring athletes. Subscribe To Newsletters. BETA. THIS IS A BETA EXPERIENCE.'}, {'url': 'https://olympics.com/en/paris-2024/videos/list/highlights', 'content': "Watch the latest highlights from P

The report on the highlights of the Paris Olympics 2024 has been successfully created and saved as `report-olympics-2024.md`. If you need any further assistance or modifications, feel free to ask!

In [48]:
with open('report-olympics-2024.md', 'r') as f:
    content_report = f.read()
    
Markdown(content_report)

# Highlights of the Paris Olympics 2024

The Paris Olympics 2024, scheduled from July 26 to August 11, 2024, promises to be a spectacular event showcasing the world's top athletes. Here are the main highlights expected from the Games:

## Key Dates and Events
- **Opening Ceremony**: July 26, 2024
- **Closing Ceremony**: August 11, 2024

## Notable Sports and Competitions
- **Basketball**: Anticipated thrilling matches featuring top international teams.
- **Tennis**: High-stakes matches with elite players competing for gold.
- **Handball**: Denmark's men's team is expected to defend their title after a strong performance in previous tournaments.

## Record-Breaking Moments
- The Games are expected to feature numerous record-breaking performances, with athletes aiming to surpass previous Olympic records.

## Inspiring Athletes
- The event will highlight inspiring stories of athletes overcoming challenges to compete at the highest level.

## Cultural Significance
- The Paris Olympics will not only focus on sports but also celebrate French culture and heritage, making it a unique experience for attendees and viewers worldwide.

## Conclusion
The Paris Olympics 2024 is set to be a historic event filled with excitement, competition, and unforgettable moments. Stay tuned for daily highlights and updates as the Games unfold.